# `pymathutils.mesh` submodule tests

In [ ]:
import sys
from pathlib import Path

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))

# ply_dir = (proj_dir / "data" / "example_ply").resolve()
ply_dir = (proj_dir / "data" / "example_ply_jan_15_26").resolve()
out_dir = (proj_dir / "output").resolve()

hex_patch_he_ply = f"{ply_dir}/hex_patch_he.ply"
hex_patch_vf_ply = f"{ply_dir}/hex_patch_vf.ply"
annulus_he_ply = f"{ply_dir}/annulus_he.ply"
annulus_vf_ply = f"{ply_dir}/annulus_vf.ply"
plane_6_he_ply = f"{ply_dir}/nice_plane/plane_0000006_he.ply"
plane_24_ply = f"{ply_dir}/nice_plane/plane_0000024_he.ply"
dumbbell_coarse_he_ply = f"{ply_dir}/dumbbell_coarse_he.ply"


## Input/Output

### `C++` $\rightarrow$ `Python` bindings

- `load_mesh_samples_from_ply`
- `write_mesh_samples_to_ply`
- `load_mesh_samples`
- `save_mesh_samples`

#### `write_mesh_samples_to_ply` zero-length array bug

Works unless: array length is zero and dtype is int

In [ ]:
import sys
from pathlib import Path
import os

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))
in_ply_dir = (proj_dir / "data" / "example_ply").resolve()
out_ply_dir = (proj_dir / "output" / "example_ply").resolve()
os.system(f"mkdir -p {out_ply_dir}")

from pymathutils.mesh import load_mesh_samples_from_ply, write_mesh_samples_to_ply
import numpy as np

inply = f"{in_ply_dir}/nice_plane/plane_0000006_he.ply"
outply = f"{out_ply_dir}/plane_0000006_he.ply"

s0 = load_mesh_samples_from_ply(inply)
s0["h_negative_B"] = np.zeros((1,), dtype=int) # nonzero length, dtype=int
write_mesh_samples_to_ply(s0, outply)
s0 = load_mesh_samples_from_ply(inply)
s0["h_negative_B"] = np.zeros((0,), dtype=np.intc) # zero length, dtype=np.intc
write_mesh_samples_to_ply(s0, outply)
s0 = load_mesh_samples_from_ply(inply)
s0["h_negative_B"] = np.zeros((0,), dtype=np.int32) # zero length, dtype=np.int32
write_mesh_samples_to_ply(s0, outply)
print("those were fine")

print("this is not")
s0 = load_mesh_samples_from_ply(inply)
s0["h_negative_B"] = np.zeros((0,), dtype=int) # zero length, dtype=int
write_mesh_samples_to_ply(s0, outply)
print("actually, just kidding. that was fine too")

#### Combined save/load tests `load_mesh_samples_from_ply`, `write_mesh_samples_to_ply`

In [ ]:
import sys
from pathlib import Path
import os

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))
in_ply_dir = (proj_dir / "data" / "example_ply").resolve()
out_ply_dir = (proj_dir / "output" / "example_ply").resolve()
os.system(f"mkdir -p {out_ply_dir}")

from pymathutils.mesh import load_mesh_samples_from_ply, write_mesh_samples_to_ply
import numpy as np

inply = f"{in_ply_dir}/dumbbell_coarse_he.ply"
outply = f"{out_ply_dir}/dumbbell_coarse_he.ply"

# inply = f"{proj_dir}/data/example_ply/annulus_he.ply"
# outply = f"{proj_dir}/output/annulus_he.ply"

print("\nTest raw ply")
s0 = load_mesh_samples_from_ply(inply)
print(f"{s0.keys()=}")
write_mesh_samples_to_ply(s0, outply)
s1 = load_mesh_samples_from_ply(outply)
print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
shapes0 = [s0[k].shape for k in s0.keys()]
shapes1 = [s0[k].shape for k in s0.keys()]
assert np.all([a==b for a,b in zip(shapes0, shapes1)])
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0

print("\nTest saved ply")
s0 = s1
print(f"{s0.keys()=}")
write_mesh_samples_to_ply(s0, outply)
s1 = load_mesh_samples_from_ply(outply)
print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0


#### Combined save/load tests `load_mesh_samples`, `save_mesh_samples`

Still crashes sometimes? Hard to reproduce consistently. Seems kinda random.

In [8]:
import sys
from pathlib import Path
import os

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))
in_ply_dir = (proj_dir / "data" / "example_ply_feb_21_26").resolve()
out_ply_dir = (proj_dir / "output" / "example_ply").resolve()
os.system(f"mkdir -p {out_ply_dir}")

from pymathutils.mesh import load_mesh_samples, save_mesh_samples
import numpy as np


inply = f"{in_ply_dir}/dumbbell_coarse_he.ply"
outply = f"{out_ply_dir}/dumbbell_coarse_he.ply"

# inply = f"{in_ply_dir}/annulus_he.ply"
# outply = f"{out_ply_dir}/annulus_he.ply"
def compare_save_load_test(inply, outply):
    print("\n"+25*"-"+"\ncompare_save_load_test")
    print("\nTest raw ply")
    s0 = load_mesh_samples(inply)
    print(f"{s0.keys()=}")
    save_mesh_samples(s0, outply)
    s1 = load_mesh_samples(outply)
    print(f"{s1.keys()=}")
    assert s0.keys() == s1.keys()
    shapes0 = [s0[k].shape for k in s0.keys()]
    shapes1 = [s0[k].shape for k in s0.keys()]
    assert np.all([a==b for a,b in zip(shapes0, shapes1)])
    diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
    diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
    print(f"{diffs0=}")
    print(f"{diffs1=}")
    assert diffs0 == 0
    assert diffs1 == 0
    
    print("\nTest saved ply")
    s0 = s1
    print(f"{s0.keys()=}")
    save_mesh_samples(s0, outply)
    s1 = load_mesh_samples(outply)
    print(f"{s1.keys()=}")
    assert s0.keys() == s1.keys()
    shapes0 = [s0[k].shape for k in s0.keys()]
    shapes1 = [s0[k].shape for k in s0.keys()]
    assert np.all([a==b for a,b in zip(shapes0, shapes1)])
    diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
    diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
    print(f"{diffs0=}")
    print(f"{diffs1=}")
    assert diffs0 == 0
    assert diffs1 == 0

def check_face_cycle_list_key():
    print("\n"+25*"-"+"\ncheck_face_cycle_list_key")
    inply = f"{in_ply_dir}/dumbbell_coarse_vf.ply"
    outply = f"{out_ply_dir}/dumbbell_coarse_vf.ply"

    print("\nTest raw ply")
    s0 = load_mesh_samples(inply)
    print(f"{s0.keys()=}")
    save_mesh_samples(s0, outply)
    s1 = load_mesh_samples(outply)
    print(f"{s1.keys()=}")
    assert s0.keys() == s1.keys()

    assert "V_cycle_F" in s0.keys()
    

compare_save_load_test(inply, outply)
check_face_cycle_list_key()


-------------------------
compare_save_load_test

Test raw ply
s0.keys()=dict_keys(['X_ambient_V', 'f_left_H', 'h_next_H', 'h_out_V', 'h_right_F', 'h_twin_H', 'v_origin_H'])
s1.keys()=dict_keys(['X_ambient_V', 'f_left_H', 'h_next_H', 'h_out_V', 'h_right_F', 'h_twin_H', 'v_origin_H'])
diffs0=np.float64(0.0)
diffs1=np.float64(0.0)

Test saved ply
s0.keys()=dict_keys(['X_ambient_V', 'f_left_H', 'h_next_H', 'h_out_V', 'h_right_F', 'h_twin_H', 'v_origin_H'])
s1.keys()=dict_keys(['X_ambient_V', 'f_left_H', 'h_next_H', 'h_out_V', 'h_right_F', 'h_twin_H', 'v_origin_H'])
diffs0=np.float64(0.0)
diffs1=np.float64(0.0)

-------------------------
check_face_cycle_list_key

Test raw ply
s0.keys()=dict_keys(['V_cycle_F', 'X_ambient_V'])
s1.keys()=dict_keys(['V_cycle_F', 'X_ambient_V'])


#### List property key tests `load_mesh_samples`, `save_mesh_samples`

In [3]:
import sys
from pathlib import Path
import os

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))
in_ply_dir = (proj_dir / "data" / "example_ply").resolve()
out_ply_dir = (proj_dir / "output" / "example_ply").resolve()
os.system(f"mkdir -p {out_ply_dir}")

from pymathutils.mesh import load_mesh_samples, save_mesh_samples
import numpy as np


inply = f"{in_ply_dir}/dumbbell_coarse_vf.ply"
outply = f"{out_ply_dir}/dumbbell_coarse_vf.ply"

s0 = load_mesh_samples(inply, ply_property_convention="MeshBrane", verbose=True)
save_mesh_samples(s0, outply)
s1 = load_mesh_samples(outply, ply_property_convention="MeshBrane")
print(f"{s0.keys()=}")
s0


	[ply_header] Type: ascii
	[ply_header] Comment: zipper output
	[ply_header] Comment: modified by flipply
	[ply_header] element: vertex (35947)
		[ply_header] property: x
		[ply_header] property: y
		[ply_header] property: z
		[ply_header] property: confidence
s0.keys()=dict_keys(['V_cycle_F', 'xyz_coord_V'])
		[ply_header] property: intensity
	[ply_header] element: face (69451)
		[ply_header] property: vertex_indices
	[ply_load] skipping property spec h_directed_E because element edge not found in file.
	[ply_load] skipping property spec h_directed_E because property h not found in file edge.
	[ply_load] skipping property spec h_out_V because property h not found in file vertex.
	[ply_load] skipping property spec h_right_F because property h not found in file face.
	[ply_load] skipping property spec V_cycle_E because element edge not found in file.
	[ply_load] skipping property spec V_cycle_E because property vertex_indices not found in file edge.
	[ply_load] skipping property spec h_

{'V_cycle_F': array([[20399, 21215, 21216],
        [14838,  9280,  9186],
        [ 5187, 13433, 16020],
        ...,
        [17279, 34909, 17346],
        [17277, 17346, 34909],
        [17345, 17346, 17277]], shape=(69451, 3)),
 'xyz_coord_V': array([[ 5.53280416e-10, -8.31918753e-14,  1.32805068e-24],
        [ 1.56426493e-09, -4.53040684e-24,  4.19373687e-16],
        [ 5.15580351e-10, -8.23871354e-16,  1.20898601e-20],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
       shape=(35947, 3))}

ound in file half_edge.
	[ply_load] skipping property spec e_undirected_H because element half_edge not found in file.
	[ply_load] skipping property spec e_undirected_H because property e not found in file half_edge.
	[ply_load] skipping property spec f_left_H because element half_edge not found in file.
	[ply_load] skipping property spec f_left_H because property f not found in file half_edge.
	[ply_load] skipping property spec h_next_H because element half_edge not found in file.
	[ply_load] skipping property spec h_next_H because property n not found in file half_edge.
	[ply_load] skipping property spec h_twin_H because element half_edge not found in file.
	[ply_load] skipping property spec h_twin_H because property t not found in file half_edge.


### HalfEdgeMesh tests

In [6]:
import sys
from pathlib import Path
import os

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))
in_ply_dir = (proj_dir / "data" / "example_ply_feb_21_26").resolve()
out_ply_dir = (proj_dir / "output" / "example_ply").resolve()
os.system(f"mkdir -p {out_ply_dir}")

from pymathutils.mesh import load_mesh_samples, save_mesh_samples, HalfEdgeTopology, HalfEdgeMesh
import numpy as np

inply = f"{in_ply_dir}/bunny.ply"
outply = f"{out_ply_dir}/annulus_he.ply"

for _ in range(100):
    s0 = load_mesh_samples(inply)
    # s0["X_ambient_V"] = s0["xyz_coord_V"] 
    m = HalfEdgeMesh()
    
    m.from_mesh_samples(s0)
    
    m.to_mesh_samples()
    m.topo.h_out_V()
    m.X_ambient_V()
    x =m.X_ambient_v(0)
    x*=2
    m.X_ambient_V()
m.topo.num_faces()
m.to_mesh_samples()

m = HalfEdgeMesh()
m.load_ply(inply, ply_property_convention="MathUtils")

m.to_mesh_samples()

m.to_mesh_samples()

{'V_cycle_F': array([[20399, 21215, 21216],
        [14838,  9280,  9186],
        [ 5187, 13433, 16020],
        ...,
        [17279, 34909, 17346],
        [17277, 17346, 34909],
        [17345, 17346, 17277]], shape=(69451, 3)),
 'X_ambient_V': array([[ 5.53280416e-10, -8.31918753e-14,  1.32805068e-24],
        [ 1.56426493e-09, -4.53040684e-24,  4.19373687e-16],
        [ 5.15580351e-10, -8.23871354e-16,  1.20898601e-20],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
       shape=(35947, 3))}

### `pymathutils.mesh.pyutils` Python implementations

- `HalfEdgeMesh`
  - `load_ply`
  - `save_ply`

In [ ]:
import sys
from pathlib import Path

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))

# ply_dir = (proj_dir / "data" / "example_ply").resolve()
ply_dir = (proj_dir / "data" / "example_ply_jan_15_26").resolve()
out_dir = (proj_dir / "output").resolve()

from pymathutils.mesh.pyutils import HalfEdgeMesh
import numpy as np
from pymathutils.mesh import load_mesh_samples_from_ply, write_mesh_samples_to_ply

# inply = f"{ply_dir}/hex_patch_he.ply"
inply = f"{ply_dir}/dumbbell_coarse_he.ply"

outply = f"{proj_dir}/output/dumbbell_coarse_he.ply"


print("\nTest raw ply")
m0 = HalfEdgeMesh.load_ply(inply)
s0 = m0.mesh_samples
print(f"{s0.keys()=}")
for k,v in s0.items():
    print(f"{k}: {v.dtype=}")
s0["h_negative_B"] = np.array([0], dtype=int)
write_mesh_samples_to_ply(s0, outply)
m0.save_ply(outply)

In [ ]:
from pymathutils.mesh.pyutils import HalfEdgeMesh
import numpy as np

inply = dumbbell_coarse_he_ply
outply = f"{proj_dir}/output/dumbbell_coarse_he.ply"


print("\nTest raw ply")
m0 = HalfEdgeMesh.load_ply(inply)
s0 = m0.mesh_samples
print(f"{s0.keys()=}")
m0.save_ply(outply)
m1 = HalfEdgeMesh.load_ply(outply)
s1 = m1.mesh_samples

print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0

print("\nTest saved ply")
m0 = m1
s0 = m0.mesh_samples
print(f"{s0.keys()=}")
m1 = HalfEdgeMesh.load_ply(outply)
s1 = m1.mesh_samples

print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0

## Visualization

### `pymathutils.mesh` core implementations

### `pymathutils.mesh.pyutils` Python implementations

In [7]:

def plot_hem0(m):
    pts_V, vecs_V = get_half_edge_vector_field(m)

    pv_m = pv.PolyData(m.xyz_coord_V, faces=np.hstack([np.full((len(m.V_cycle_F), 1), 3), m.V_cycle_F]))
    
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(pv_m, color="lightblue", show_edges=True)
    plotter.add_arrows(pts_V, vecs_V, mag=1.0, color="red")
    plotter.show(jupyter_backend='trame')
    return plotter

def plot_hem(m):
    import pyvista as pv
    from mesh_viewer import get_half_edge_vector_field
    import numpy as np
    # Define PyVista surface mesh
    pv_m = pv.PolyData(m.xyz_coord_V, faces=np.hstack([np.full((len(m.V_cycle_F), 1), 3), m.V_cycle_F]))
    # Define PyVista half-edge vector field
    pts_H, vecs_H = get_half_edge_vector_field(m)
    vecs_H_source = pv.Arrow(
    tip_length=0.125,      # Arrow tip length (0.0-1.0)
    tip_radius=0.025,       # Arrow tip radius
    tip_resolution=5,    # Tip smoothness
    shaft_radius=0.00625,   # Shaft thickness
    shaft_resolution=3,   # Shaft smoothness
    )
    pv_pts_H = pv.PolyData(pts_H)
    pv_pts_H['half_edges'] = vecs_H
    pv_H = pv_pts_H.glyph(orient='half_edges', scale='half_edges', factor=1, geom=vecs_H_source,)
    
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(pv_m, color="lightblue", show_edges=True)
    plotter.add_mesh(pv_H, color="red")
    plotter.show(jupyter_backend='trame')
    return plotter